# Machine Learning Translation
In this project we aim to convert English phrases to French using RNN on Deep Learning Neural Network

# import libraries :

In [2]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow as tf
import os
import collections
import re

# Read and preprocess data :

[english](https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_en.txt)

[french](https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_fr.txt)

In [3]:
english_path='https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_en.txt'
french_path='https://raw.githubusercontent.com/projjal1/datasets/master/small_vocab_fr.txt'

#### Load the dataset and split file by lines

In [4]:
import os

def load_data(path):
  input_file = os.path.join(path)
  with open(input_file, "r") as f:
    data = f.read()

  return data.split('\n')

In [5]:
english_data=tf.keras.utils.get_file('file1',english_path)
french_data=tf.keras.utils.get_file('file2',french_path)

In [6]:
english_sentences=load_data(english_data)
french_sentences=load_data(french_data)


#### few examples 

In [7]:
for i in range(5):
  print('Sample :',i)
  print(english_sentences[i])
  print(french_sentences[i])
  print('-'*50)

Sample : 0
new jersey is sometimes quiet during autumn , and it is snowy in april .
new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
--------------------------------------------------
Sample : 1
the united states is usually chilly during july , and it is usually freezing in november .
les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .
--------------------------------------------------
Sample : 2
california is usually quiet during march , and it is usually hot in june .
california est gÃ©nÃ©ralement calme en mars , et il est gÃ©nÃ©ralement chaud en juin .
--------------------------------------------------
Sample : 3
the united states is sometimes mild during june , and it is cold in september .
les Ã©tats-unis est parfois lÃ©gÃ¨re en juin , et il fait froid en septembre .
--------------------------------------------------
Sample : 4
your least liked fruit is the grape , but my least liked is the apple .
votre moin

#### Convert to Vocabulary

In [8]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('English Vocab:',len(english_words_counter))
print('French Vocab:',len(french_words_counter))
print(english_words_counter)

English Vocab: 227
French Vocab: 355
Counter({'is': 205858, ',': 140897, '.': 129039, 'in': 75525, 'it': 75137, 'during': 74933, 'the': 67628, 'but': 63987, 'and': 59850, 'sometimes': 37746, 'usually': 37507, 'never': 37500, 'least': 27564, 'favorite': 27371, 'fruit': 27105, 'most': 14934, 'loved': 13666, 'liked': 13546, 'new': 12197, 'paris': 11334, 'india': 11277, 'united': 11270, 'states': 11270, 'california': 11250, 'jersey': 11225, 'france': 11170, 'china': 10953, 'he': 10786, 'she': 10786, 'grapefruit': 10118, 'your': 9734, 'my': 9700, 'his': 9700, 'her': 9700, 'fall': 9134, 'june': 9133, 'spring': 9102, 'january': 9090, 'winter': 9038, 'march': 9023, 'autumn': 9004, 'may': 8995, 'nice': 8984, 'september': 8958, 'july': 8956, 'april': 8954, 'november': 8951, 'summer': 8948, 'december': 8945, 'february': 8942, 'our': 8932, 'their': 8932, 'freezing': 8928, 'pleasant': 8916, 'beautiful': 8915, 'october': 8910, 'snowy': 8898, 'warm': 8890, 'cold': 8878, 'wonderful': 8808, 'dry': 8794

# Tokenize 
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings. Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number. These are called character and word ids, respectively. Character ids are used for character level models that generate text predictions for each character. A word level model uses word ids that generate text predictions for each word. Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's Tokenizer function. Use this function to tokenize english_sentences and french_sentences in the cell below.

In [9]:
def tokenize(x):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(x)
  return tokenizer.texts_to_sequences(x), tokenizer

#### Sample

In [10]:
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
  print('Sequence {} in x'.format(sample_i + 1))
  print('  Input:  {}'.format(sent))
  print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


# Padding 
When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the end of each sequence using Keras's pad_sequences function.

In [11]:
def pad(x, length=None):
  return pad_sequences(x, maxlen=length, padding='post')

In [12]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    #Expanding dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


## Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want. We want the French translation. The function logits_to_text will bridge the gab between the logits from the neural network to the French translation. 

In [13]:
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  #So basically we are predicting output for a given word and then selecting best answer
  #Then selecting that label we reverse-enumerate the word from id
  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

![Model](https://github.com/tommytracey/AIND-Capstone/raw/8267d4fe72e48c595a0aff46eaf0a805fff0f36d/images/embedding.png)

# Building Model
Here we use RNN model combined with GRU nodes for translation

In [13]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    input_shape: Tuple of input shape
    output_sequence_length: Length of output sequence
    english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters نرخ یادگیری بهینه‌ساز Adam
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    # تبدیل کلمات انگلیسی به بردارهای 256 بعدی
    model.add(GRU(256, return_sequences=True))  
    # لایه بازگشتی با 256 واحد حافظه و حفظ خروجی برای هر تایم‌استپ
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    # لایه متراکم 1024 واحدی با ReLU برای هر تایم‌استپ
    model.add(Dropout(0.5))
    # جلوگیری از اورفیتینگ با غیرفعال کردن تصادفی 50% نورون‌ها
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 
    # لایه خروجی که برای هر کلمه، احتمال تمام کلمات فرانسوی را محاسبه می‌کند
    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [14]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

## Finally calling the model function

### 1. پارامترهای ورودی تابع embed_model :
tmp_x.shape: شکل داده‌های ورودی انگلیسی (بعد از پیش‌پردازش)

معمولاً به شکل (تعداد جملات, حداکثر طول جمله) است

preproc_french_sentences.shape[1]: طول جملات فرانسوی (پس از padding)

len(english_tokenizer.word_index)+1: اندازه دایره لغات انگلیسی + ۱ (برای padding)

len(french_tokenizer.word_index)+1: اندازه دایره لغات فرانسوی + ۱ (برای padding)

### 2. ساختار مدل ایجاد شده:
مدل ایجاد شده شامل این لایه‌هاست  :

لایه Embedding:

تبدیل کلمات انگلیسی به بردارهای متراکم
مثال: "cat" → [0.2, -0.5, 0.7, ...] (بردار 256 بعدی)

لایه GRU:

256 واحد حافظه

return_sequences=True برای حفظ خروجی در هر گام زمانی

لایه‌های Dense با TimeDistributed :

پردازش مستقل هر گام زمانی

لایه نهایی با softmax برای پیش‌بینی احتمالات کلمات فرانسوی

مثال :
"the weather is nice"

تبدیل هر کلمه به بردار Embedding

پردازش توسط GRU

تولید توزیع احتمال برای هر کلمه فرانسوی در خروجی

In [15]:
simple_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

In [16]:
simple_rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 21, 256)           51200     
                                                                 
 gru (GRU)                   (None, 21, 256)           394752    
                                                                 
 time_distributed (TimeDist  (None, 21, 1024)          263168    
 ributed)                                                        
                                                                 
 dropout (Dropout)           (None, 21, 1024)          0         
                                                                 
 time_distributed_1 (TimeDi  (None, 21, 346)           354650    
 stributed)                                                      
                                                                 
Total params: 1063770 (4.06 MB)
Trainable params: 106377

# Training the model
Here we start to train the model and pass the english text and the max_sequence_length, with vocab size for both english and french text

In [17]:
history=simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

Epoch 1/20


108/108 [==============================] - 85s 771ms/step - loss: 1.2600 - accuracy: 0.7007 - val_loss: 0.4460 - val_accuracy: 0.8564
Epoch 2/20
108/108 [==============================] - 85s 788ms/step - loss: 0.3841 - accuracy: 0.8739 - val_loss: 0.2872 - val_accuracy: 0.9031
Epoch 3/20
108/108 [==============================] - 85s 786ms/step - loss: 0.2782 - accuracy: 0.9071 - val_loss: 0.2303 - val_accuracy: 0.9219
Epoch 4/20
108/108 [==============================] - 86s 799ms/step - loss: 0.2352 - accuracy: 0.9200 - val_loss: 0.2134 - val_accuracy: 0.9260
Epoch 5/20
108/108 [==============================] - 84s 782ms/step - loss: 0.2130 - accuracy: 0.9267 - val_loss: 0.1948 - val_accuracy: 0.9326
Epoch 6/20
108/108 [==============================] - 86s 793ms/step - loss: 0.2005 - accuracy: 0.9301 - val_loss: 0.1921 - val_accuracy: 0.9329
Epoch 7/20
108/108 [==============================] - 86s 795ms/step - loss: 0.1925 - accuracy: 0.9324 - val_loss: 0.1840 - val_

In [18]:
simple_rnn_model.save('model.h5')

C:\app sorc\ana\envs\test2\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#  Predictions


In [19]:
def final_predictions(text):
  y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
  y_id_to_word[0] = '<PAD>'

  sentence = [english_tokenizer.word_index[word] for word in text.split()]
  sentence = pad_sequences([sentence], maxlen=preproc_french_sentences.shape[-2], padding='post')
  
  print(sentence.shape)
  print(logits_to_text(simple_rnn_model.predict(sentence[:1])[0], french_tokenizer))

In [31]:
txt=input().lower()
final_predictions(re.sub(r'[^\w]', ' ', txt))

 he likes grape


(1, 21)
1/1 [==============================] - 0s 21ms/step
il aime le raisin <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
